In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# !git clone https://github.com/WongKinYiu/yolov7.git

path = ''
!unzip "{path}"yolov7_det.zip

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolov7_det.zip
   creating: yolov7_det/
   creating: yolov7_det/paper/
  inflating: yolov7_det/LICENSE.md   
   creating: yolov7_det/tools/
  inflating: yolov7_det/.DS_Store    
  inflating: __MACOSX/yolov7_det/._.DS_Store  
  inflating: yolov7_det/requirements.txt  
   creating: yolov7_det/deploy/
   creating: yolov7_det/figure/
   creating: yolov7_det/utils/
   creating: yolov7_det/models/
  inflating: yolov7_det/test.py      
  inflating: yolov7_det/export.py    
   creating: yolov7_det/cfg/
  inflating: yolov7_det/README.md    
  inflating: yolov7_det/train_aux.py  
  inflating: yolov7_det/.gitignore   
   creating: yolov7_det/scripts/
  inflating: yolov7_det/train.py     
   creating: yolov7_det/inference/
  inflating: yolov7_det/hubconf.py   
   creating: yolov7_det/.git/
   creating: yolov7_det/data/
  inflating: yolov7_det/detect.py    
  inflating: yolov7_det/paper/yolov7.pdf  
  inflating: yolov7_det/tools

In [13]:
%cd yolov7_det

/content/yolov7_det


In [14]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.1 MB/s eta 0:00:00


In [5]:
### based on: https://blog.paperspace.com/train-yolov7-custom-data/
import torch
from IPython.display import Image
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [6]:
!unzip "{path}"yolo_bbox_data.zip -d "/content/"

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolo_bbox_data.zip
   creating: /content/yolo_data/
   creating: /content/yolo_data/images/
  inflating: /content/yolo_data/images/144.png  
  inflating: /content/yolo_data/images/624.png  
  inflating: /content/yolo_data/images/55.png  
  inflating: /content/yolo_data/images/930.png  
  inflating: /content/yolo_data/images/774.png  
  inflating: /content/yolo_data/images/439.png  
  inflating: /content/yolo_data/images/226.png  
  inflating: /content/yolo_data/images/177.png  
  inflating: /content/yolo_data/images/552.png  
  inflating: /content/yolo_data/images/572.png  
  inflating: /content/yolo_data/images/868.png  
  inflating: /content/yolo_data/images/243.png  
  inflating: /content/yolo_data/images/110.png  
  inflating: /content/yolo_data/images/318.png  
  inflating: /content/yolo_data/images/196.png  
  inflating: /content/yolo_data/images/205.png  
  inflating: /content/yolo_data/images/466.png  
  inf

In [7]:
# create folders for the data after splitting
!mkdir split_data
!mkdir split_data/images
!mkdir split_data/labels
!mkdir split_data/images/train
!mkdir split_data/labels/train
!mkdir split_data/images/val
!mkdir split_data/labels/val
!mkdir split_data/images/test
!mkdir split_data/labels/test

In [8]:
ims_path = '/content/yolo_data/images/'
annot_path = '/content/yolo_data/labels/'
images = [os.path.join(ims_path, x) for x in os.listdir(ims_path)]
annotations = [os.path.join(annot_path, x) for x in os.listdir(annot_path) if x[-3:] == "txt"]

In [9]:
images.sort()
annotations.sort()

train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [10]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, 'split_data/images/train')
move_files_to_folder(val_images, 'split_data/images/val/')
move_files_to_folder(test_images, 'split_data/images/test/')
move_files_to_folder(train_annotations, 'split_data/labels/train/')
move_files_to_folder(val_annotations, 'split_data/labels/val/')
move_files_to_folder(test_annotations, 'split_data/labels/test/')

In [ ]:
import multiprocessing
workers = multiprocessing.cpu_count()
print("number of workers: ", workers)

In [11]:
!python train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.myCustom.yaml --batch 32 --epochs 600 --data data/data.yaml --weights '' --workers 12 --name yolo_detection

python3: can't open file '/content/train.py': [Errno 2] No such file or directory


##### inference

In [ ]:
!python detect.py --source split_data/images/test/ --weights runs/train/yolo_detection/weights/best.pt --conf 0.25 --name yolo_detection

In [ ]:
detections_dir = "runs/detect/yolo_detection/"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))

##### test

In [ ]:
!python test.py --weights runs/train/yolo_detection/weights/best.pt --data data.yaml --task test --name yolo_detection

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='./data/data.yaml', device='', exist_ok=False, img_size=640, iou_thres=0.65, name='yolo_detection', no_trace=False, project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='test', v5_metric=False, verbose=False, weights=['runs/train/yolo_detection/weights/best.pt'])
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536.125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36514136 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Trace

In [ ]:
!zip -r detection_ims.zip /content/yolov7_det/runs/detect/yolo_detection
!zip -r stats.zip /content/yolov7_det/runs/test/yolo_detection
!zip -r train.zip /content/yolov7_det/runs/train/yolo_detection/